In [1]:
%time
import sys
import re
import math
import requests
import enlighten

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

url = 'http://200.152.38.155/CNPJ/'
formato_data = '%Y%m%d'
PATH_BD = 'data/CNPJ_full.db'

Wall time: 0 ns


In [2]:
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip"], list_urls))
tamanho = len(filtered)
MANAGER = enlighten.get_manager()
bara_lista = MANAGER.counter(total=tamanho, desc='Arquivos', unit='Arquivos')

In [ ]:
print("")
print(f"Fazendo download dos {tamanho} arquivos encontrados.", flush=False)
print("")


for indice, elemento in enumerate(filtered):
    bara_lista.update()
    
    r = requests.get(url + elemento, stream=True, headers={"Accept-Encoding": None})
    if r.status_code == requests.codes.ok:
        assert r.status_code == 200, r.status_code
        dlen = int(r.headers.get('Content-Length', '0')) or None
        barra_download = MANAGER.counter(color = 'green', desc= f'Arquivo {indice + 1} de {tamanho}', 
                             total = dlen and math.ceil(dlen / 2 ** 20), 
                             unit = 'MiB', leave = False)
        with barra_download as ctr, \
             open("zips/" + elemento, 'wb', buffering = 2 ** 24) as f:
            for chunk in r.iter_content(chunk_size = 2 ** 20):
                # print(chunk[-16:].hex().upper())
                print(f'\r{elemento} ------ Está sendo baixado!!!', end='', flush=True)
                sys.stdout.flush()
                f.write(chunk)
                ctr.update()
        
    else:
        r.raise_for_status()
print(f'\r Download Concluido!!!', end='', flush=True)
MANAGER.stop()

In [3]:
%time
list_estabelecimentos = [
"cnpj", 
"cnpj_ordem", 
"cnpj_dv", 
"matriz_filial",
"nome",
"situacao", 
"data_situacao", 
"motivo_situacao", 
"cidade_exterior", 
"pais", 
"inicio_atividade", 
"cnae_fiscal", 
"cnae_secundario", 
"tipo_logradouro", 
"logradouro", 
"numero", 
"complemento", 
"bairro", 
"cep", 
"uf", 
"municipio", 
"ddd_1", 
"telefone_1", 
"ddd_2", 
"telefone_2", 
"ddd_fax", 
"numero_fax", 
"email", 
"situacao_especial", 
"data_situacao_especial"
]

Wall time: 0 ns


In [4]:
%time

import sqlite3

connection = sqlite3.connect(PATH_BD)
con = connection.cursor()

sql = """CREATE TABLE IF NOT EXISTS ESTABELECIMENTOS 
                    (CNPJ                   VARCHAR (8),
                    CNPJ_ORDEM             VARCHAR (4),
                    CNPJ_DV                VARCHAR (2),
                    MATRIZ_FILIAL          INTEGER,
                    NOME                   VARCHAR,
                    SITUACAO               INTEGER,
                    DATA_SITUACAO          DATE,
                    MOTIVO_SITUACAO        VARCHAR,
                    CIDADE_EXTERIOR        VARCHAR,
                    PAIS                   INTEGER,
                    INICIO_ATIVIDADE       DATE,
                    CNAE_FISCAL            VARCHAR,
                    CNAE_SECUNDARIO        VARCHAR,
                    TIPO_LOGRADOURO        VARCHAR,
                    LOGRADOURO             VARCHAR,
                    NUMERO                 VARCHAR,
                    COMPLEMENTO            VARCHAR,
                    BAIRRO                 VARCHAR,
                    CEP                    VARCHAR (8),
                    UF                     VARCHAR (2),
                    MUNICIPIO              INTEGER,
                    DDD_1                  VARCHAR,
                    TELEFONE_1             VARCHAR (11),
                    DDD_2                  INTEGER,
                    TELEFONE_2             VARCHAR (11),
                    DDD_FAX                INTEGER,
                    NUMERO_FAX             VARCHAR (11),
                    EMAIL                  VARCHAR,
                    SITUACAO_ESPECIAL      VARCHAR,
                    DATA_SITUACAO_ESPECIAL DATE 
                    )
                """


con.execute('DROP TABLE IF EXISTS ESTABELECIMENTOS') 
con.execute('VACUUM') 
con.execute(sql)


Wall time: 0 ns


In [6]:
lista = [l for l in filtered if "ESTABELE" in l]

for item in lista:
    print(item)
    df = pd.read_csv("zips/" + item, 
                     compression='zip',
                     header=None, 
                     sep=';', 
                     quotechar='"', 
                     encoding = "ISO-8859-1", 
                     low_memory=False,
                     dtype = {'cnpj': str, 'cnpj_ordem': str, 'cnpj_dv': str},
                     names=list_estabelecimentos,
                     parse_dates=['data_situacao','inicio_atividade','data_situacao_especial'])


#    df['data_situacao']= df['data_situacao'].replace(0, np.nan)

#    df['data_situacao'] = pd.to_datetime(df['data_situacao'], format=formato_data)
#    df['inicio_atividade'] = pd.to_datetime(df['inicio_atividade'], format=formato_data)
#    df['data_situacao_especial'] = pd.to_datetime(df['data_situacao_especial'], format=formato_data)

    df.to_sql(name = 'ESTABELECIMENTOS', con = connection, if_exists = 'append', index = False)

K3241.K03200Y0.D10710.ESTABELE.zip
K3241.K03200Y1.D10710.ESTABELE.zip
K3241.K03200Y2.D10710.ESTABELE.zip
K3241.K03200Y3.D10710.ESTABELE.zip
K3241.K03200Y4.D10710.ESTABELE.zip
K3241.K03200Y5.D10710.ESTABELE.zip
K3241.K03200Y6.D10710.ESTABELE.zip
K3241.K03200Y7.D10710.ESTABELE.zip
K3241.K03200Y8.D10710.ESTABELE.zip
K3241.K03200Y9.D10710.ESTABELE.zip


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4753435 entries, 0 to 4753434
Data columns (total 30 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   cnpj                    object        
 1   cnpj_ordem              object        
 2   cnpj_dv                 object        
 3   matriz_filial           int64         
 4   nome                    object        
 5   situacao                int64         
 6   data_situacao           object        
 7   motivo_situacao         int64         
 8   cidade_exterior         object        
 9   pais                    float64       
 10  inicio_atividade        object        
 11  cnae_fiscal             int64         
 12  cnae_secundario         object        
 13  tipo_logradouro         object        
 14  logradouro              object        
 15  numero                  object        
 16  complemento             object        
 17  bairro                  object        
 18  ce

In [13]:
back = pd.read_sql('select * from estabelecimentos', connection)

back

MemoryError: 

In [ ]:
df = pd.DataFrame(filtered)
df[df[0].str.contains("EMPRECSV", regex=False)]


In [ ]:
lista = [l for l in filtered if "EMPRECSV" in l]
lista

In [ ]:
filtered

In [ ]:
list_tabelas = [
                "SIMPLES", 
                "CNAECSV",
                "MOTICSV",
                "MUNICCSV",
                "NATJUCSV",
                "PAISCSV",
                "QUALSCSV",
                "EMPRECSV",
                "ESTABELE",
                "SOCIOCSV",
                "EMPRECSV"]



In [ ]:
for l in list_tabelas:
    lista = [item for item in filtered if l in item]
    display(lista)

In [ ]:

dtypes = np.dtype(
    [
        ("a", str),
        ("b", int),
        ("c", float),
        ("d", np.datetime64),
        
    ]
)
df = pd.DataFrame(np.empty(0, dtype=dtypes))

In [ ]:
encoding_list = ['ascii', 'big5', 'big5hkscs', 'cp037', 'cp273', 'cp424', 'cp437', 'cp500', 'cp720', 'cp737'
                 , 'cp775', 'cp850', 'cp852', 'cp855', 'cp856', 'cp857', 'cp858', 'cp860', 'cp861', 'cp862'
                 , 'cp863', 'cp864', 'cp865', 'cp866', 'cp869', 'cp874', 'cp875', 'cp932', 'cp949', 'cp950'
                 , 'cp1006', 'cp1026', 'cp1125', 'cp1140', 'cp1250', 'cp1251', 'cp1252', 'cp1253', 'cp1254'
                 , 'cp1255', 'cp1256', 'cp1257', 'cp1258', 'euc_jp', 'euc_jis_2004', 'euc_jisx0213', 'euc_kr'
                 , 'gb2312', 'gbk', 'gb18030', 'hz', 'iso2022_jp', 'iso2022_jp_1', 'iso2022_jp_2'
                 , 'iso2022_jp_2004', 'iso2022_jp_3', 'iso2022_jp_ext', 'iso2022_kr', 'latin_1', 'iso8859_2'
                 , 'iso8859_3', 'iso8859_4', 'iso8859_5', 'iso8859_6', 'iso8859_7', 'iso8859_8', 'iso8859_9'
                 , 'iso8859_10', 'iso8859_11', 'iso8859_13', 'iso8859_14', 'iso8859_15', 'iso8859_16', 'johab'
                 , 'koi8_r', 'koi8_t', 'koi8_u', 'kz1048', 'mac_cyrillic', 'mac_greek', 'mac_iceland', 'mac_latin2'
                 , 'mac_roman', 'mac_turkish', 'ptcp154', 'shift_jis', 'shift_jis_2004', 'shift_jisx0213', 'utf_32'
                 , 'utf_32_be', 'utf_32_le', 'utf_16', 'utf_16_be', 'utf_16_le', 'utf_7', 'utf_8', 'utf_8_sig']

for encoding in encoding_list:
    worked = True
    try:
        df = pd.read_csv('K3241.K03200Y0.D10710.ESTABELE.zip', compression='zip',header=None, sep=';', quotechar='"', encoding=encoding)
    except:
        worked = False
    if worked:
        print(encoding, ':\n', df.head())

In [11]:
df1 = pd.DataFrame()

In [12]:
type(df1)

pandas.core.frame.DataFrame

In [ ]:
import math
import requests, enlighten

url = 'http://200.152.38.155/CNPJ/F.K03200$W.SIMPLES.CSV.D10710.zip'
fname = 'F.K03200$W.SIMPLES.CSV.D10710.zip'

# Should be one global variable
MANAGER = enlighten.get_manager()

r = requests.get(url, stream = True)
assert r.status_code == 200, r.status_code
dlen = int(r.headers.get('Content-Length', '0')) or None

with MANAGER.counter(color = 'green', total = dlen and math.ceil(dlen / 2 ** 20), unit = 'MiB', leave = False) as ctr, \
     open(fname, 'wb', buffering = 2 ** 24) as f:
    for chunk in r.iter_content(chunk_size = 2 ** 20):
        # print(chunk[-16:].hex().upper())
        f.write(chunk)
        ctr.update()

In [ ]:
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip","pdf"], list_urls))
tamanho = len(filtered)
for indice, elemento in tqdm(enumerate(filtered)):
        resposta = requests.get(url + elemento, stream=True, headers={"Accept-Encoding": None})
        if resposta.status_code == requests.codes.ok:
            total_size = int(resposta.headers.get("Content-Length", 0))
            block_size = 1024

            print(f"Downloading {elemento}")

            with tqdm(total=total_size,file=sys.stdout) as pbar:
                with open(elemento, "wb") as f:
                    for data in resposta.iter_content(block_size):
                        pbar.set_description(f"Processing {indice +1}")
                        pbar.update(len(data))
                        f.write(data)
        else:
                resposta.raise_for_status()


In [ ]:
from tqdm import tqdm
from time import sleep

In [ ]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    sleep(0.25)
    pbar.set_description("Processing %s" % char)

In [ ]:
donloads(url, list_urls[1])

In [ ]:
def donloads(url, arquivo):
    resposta = requests.get(url + arquivo, stream=True, headers={"Accept-Encoding": None})
    if resposta.status_code == requests.codes.ok:
        total_size = int(resposta.headers.get("Content-Length", 0))
        block_size = 1024

        print(f"Downloading {arquivo}")

        t = tqdm(total=total_size, unit="iB", unit_scale=True)
        with open(arquivo, "wb") as f:
            for data in resposta.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    else:
        resposta.raise_for_status()

In [ ]:

req = requests.get(url, stream=True, headers={"Accept-Encoding": None})

In [ ]:
req.links

In [ ]:
url = 'http://200.152.38.155/CNPJ/'

In [ ]:
def baixa_arquivo(url, endereco):
    # faz a requisição 
    resposta = requests.get(url)
    if resposta.status_code == requests.codes.ok:
        with open('endereco', 'wb')as novo_arquivo:
            novo_arquivo.write(resposta.content)
        print("Download finalizado, salvo em: ()".format(endereco))
    else:
        respósta.raise_for_status()

In [ ]:
baixa_arquivo(url, 'teste.html')

In [ ]:
data = str(urlopen(URL_BASE_RFB).read(), encoding='utf8')

    # Pega todas as urls
    urls = re.findall(r'href=[\'"]?([^\'" >]+)', data)

In [ ]:
resposta = requests.get(url).text

In [ ]:
print(resposta.text)

In [ ]:
import re

In [ ]:
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)

In [ ]:
list_urls

In [ ]:
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip","pdf"], list_urls))
filtered

In [ ]:
for indice, elemento in tqdm(enumerate(filtered)):
    print (f"{indice:<10}{elemento}")

In [ ]:
def donloads(url, arquivo):
    req = requests.get(url + arquivo, stream=True, headers={"Accept-Encoding": None})
    if resposta.status_code == requests.codes.ok:
        total_size = int(req.headers.get("Content-Length", 0))
        block_size = 1024

        print(f"Downloading {arquivo}")

        t = tqdm(total=total_size, unit="iB", unit_scale=True)
        with open(arquivo, "wb") as f:
            for data in req.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    else:
        resposta.raise_for_status()

In [ ]:
for i in list_urls:
    if i.split(".")[-1] in ["zip","pdf"]:
        donloads(url, i)

In [ ]:
donloads(url, list_urls[1])